# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [65]:
!git clone https://github.com/valmirf/redes_neurais_bcc.git

fatal: destination path 'redes_neurais_bcc' already exists and is not an empty directory.


Definição da função de base radial

In [66]:
#função de base radial gaussiana
#c = centro 
#s= largura
#x = valor qualquer
def rbfGaussiana(x, c, s):
    return 1 / (np.sqrt((x-c)**2 + s **2 ))

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  stds = []
  for c1 in centers:
    dist = []
    for c2 in centers:
      if not np.array_equiv(c1,c2):
        dist.append(np.sqrt(np.sum(c1-c2) ** 2))
    dMin = np.min(dist)
    stds.append(dMin/2)  
  return np.array(stds)



2ª Etapa - Treinamento de uma Rede Neural

In [67]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=4, attnumber=18, lr=0.01, epochs=100, rbf=rbfGaussiana, computeStds=computeEqualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbf # função de base radial
        self.computeStds = computeStds  #função de cálculo da largura do campo receptivo


        self.w = np.random.randn(self.k,attnumber)
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias 
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        #print('centers: ', self.centers)
        
        #Cálculo la dargura do campo receptivo
        #2º parâmetro da rede RBF
        self.stds = self.computeStds(self.centers,self.k)
        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = self.sigmoid(F)
                out = 0 if out < 0.5 else 1
                
                #função de perda 
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))

                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                #3º Parâmetro da rede 
                self.w = self.w + self.lr * error * phi 
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            #print('F: ' + str(F))
            F = np.sum(F) + self.b
            #print('F2: ' + str(F))
            out = self.sigmoid(F)
            out = 0 if out < 0.5 else 1
            y_pred.append(out)

        return np.array(y_pred)

    # Função de Ativação Sigmóide
    def sigmoid(self, net):
      return 1.0/(1.0+np.exp(-net))




```
# Isto está formatado como código
```

# Executando a rede neural RBF

In [68]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_bcc/SOM/iris.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)
    
    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

#transforma rótulos do conjunto de treinamento em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(training_labels.values)
training_labels_transformed = le.transform(training_labels.values)

#chama RBF
rbfnet = RBFNet(lr=1e-2, attnumber=4, k=5, computeStds=computeEqualStds)
rbfnet.fit(training_inputs.values, training_labels_transformed)

#transforma rótulos do conjunto de teste em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(test_labels.values)
test_labels_transformed = le.transform(test_labels.values)

y_pred = rbfnet.predict(test_labels_transformed)
errorabs = abs(test_labels_transformed-y_pred)

print('error: ' + str(np.sum(errorabs)) + '/' + str(len(test_labels_transformed))) 
print('error: ', np.sum(errorabs)/len(test_labels_transformed)*100)

error: 20/30
error:  66.66666666666666


# Descrição Mini Projeto

**Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo iris.csv.**


## **1- Calcular a quantidade de neurônios escondidos:**

## **a) 2**

error: 20/30
error:  66.66666666666666

## **b) 3**

error: 20/30
error:  66.66666666666666

## **c) 4**

error: 20/30
error:  66.66666666666666

## **d) 5**

error: 8/30
error:  26.666666666666668


## **Qual foi a melhor configuração? Avaliaria um outro valor?**

A melhor configuração realizada foi com 5 e 6 neurônios, porém realizando a configuração com um número maior de neurônios acima ou abaixo dos valores anteriormente citados, ele comporta-se com resultados diferente dos neurônios 5 e 6 ou seja 66.66666 

o outro valor avaliado foi com 6 neuronios

## **2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial. Altere o código de forma que cada neurônio possua sua própria largura.**

Resposta: 

```
error: 8/30
error:  26.666666666666668

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  stds = []
  for c1 in centers:
    dist = []
    for c2 in centers:
      if not np.array_equiv(c1,c2):
        dist.append(np.sqrt(np.sum(c1-c2) ** 2))
    dMin = np.min(dist)
    stds.append(dMin/2)  
  return np.array(stds)
```



## **3- Modifique a função de base radial implementada (Gaussiana), para a Multiquadrática inversa e calcule a taxa de erro para as configurações das questões anteriores.**


```
def rbfGaussiana(x, c, s):
    return 1 / (np.sqrt((x-c)**2 + s **2 ))


error: 20/30
error:  66.66666666666666

```

## **Qual foi a melhor configuração?**

As melhores configurações apresentadas até o momento foram com os números de neurônios 5 e 6, com o resultado da função Gaussiana que apresentou o resultado bem abaixo dos valores dos neurônios 5 e 6 em relação a função Multiquadrática Inversa